<a href="https://colab.research.google.com/github/tomkart/musicgen-colab/blob/main/text_to_music.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip
!pip install --upgrade transformers scipy
!pip install accelerate

In [2]:
import scipy
from transformers import pipeline
from IPython.display import Audio
import numpy as np

In [3]:
# Run inference via the Transformers modelling code.
# You can use the processor + generate code to convert text into a mono 32 kHz audio waveform for more fine-grained control.
from transformers import AutoProcessor, MusicgenForConditionalGeneration

processor = AutoProcessor.from_pretrained("facebook/musicgen-medium")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-medium").to("cuda")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5Encoder

In [6]:
inputs = processor(
    text=["An 80s driving pop song with heavy drums in the background"],
    padding=True,
    return_tensors="pt",
)

# Move the input tensors to the same device as the model
inputs = inputs.to("cuda")

audio_values = model.generate(**inputs, max_new_tokens=1024)

In [7]:
sampling_rate = model.config.audio_encoder.sampling_rate
# Move the tensor to CPU before converting to NumPy array
scipy.io.wavfile.write("musicgen_med.wav", rate=sampling_rate, data=audio_values[0, 0].cpu().numpy())
Audio(audio_values[0].cpu().numpy(), rate=sampling_rate) # Move tensor to cpu before using numpy()